In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import gensim
import os
import copy
import datetime
import warnings
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from wordcloud import WordCloud
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import contextlib
import spacy
import re
import os
import random
import math
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 
tweet_tokenizer = TweetTokenizer()

In [ ]:
slang = pd.read_csv("drive/My Drive/Thesis/Data/slang-corpus.txt", sep='\s*,\s*') 
slang.head(20)

tweets= pd.read_csv("drive/My Drive/Thesis/Data/tw_data_may.csv") 
tweets.head(10000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,date,tweet
0,2021-05-01 23:59:05+00:00,Selamat pagi bro n sis. Lebih baik #stayathome...
1,2021-05-01 23:57:34+00:00,Merinding rasanya ngeliat lautan manusia di Ta...
2,2021-05-01 23:57:00+00:00,Walikota Jakarta pusat sebelumnya sudah mengin...
3,2021-05-01 23:56:26+00:00,Khofifah ingin mengawali dg peta penyebaran co...
4,2021-05-01 23:55:47+00:00,sebagai orang yg masih percaya bahwa covid mas...
...,...,...
9995,2021-05-05 08:28:01+00:00,"AS berencana untuk meminjam hampir $ 1,3 trili..."
9996,2021-05-05 08:27:35+00:00,Teman2 nakes kl ada jurnal ttg covid atau arti...
9997,2021-05-05 08:26:19+00:00,Sir corona ki wajah se hmara buisness band ho ...
9998,2021-05-05 08:25:07+00:00,#BREAKING: Filipina laporkan kasus harian Covi...


In [ ]:
class Preprocessing(object):
   
    def __init__(self):
        self.stopwords = stopwords.words('indonesian')
        self.ignored_word = ['rt','sih', 'loh', 'psx', 'ini', 'itu','eh', 'nya','si', 'oh', 'yg', 'ya', 'malaysia']
        self.append_stopwords(self.ignored_word)
            
    def lowercase(self, text):
        """ Lowercase text """
        return text.lower()
    
    def append_stopwords(self, text):
        """ Append Custom Stopword """
        for word in text:
            self.stopwords.append(word)
 
    def remove_newlines(self, text):
        """ Remove newlines characters from text """
        return text.replace("\n", "")
    
   
    def remove_tabs(self,text):
        """ Remove tab characters from text """
        return text.replace("\t", "")
    
   
    def remove_digits(self,text):
        """ Remove digits from text """
        return ''.join([word for word in text if not word.isdigit()])
    
    def remove_url(self,text):
        """ Remove urls from text """
        tokens = tweet_tokenizer.tokenize(text)
        output_array = []
        for word in tokens:
            cleaned_word = re.sub(r'^https?:\/\/.*[\r\n]*', '', word, flags=re.MULTILINE)
            output_array.append(cleaned_word)
        return " ".join(output_array)
        
        
    def replace_special_chars(self,text):
        """ Remove special characters from text """
        return re.sub(r'\W+', ' ', text)

    
    def remove_multiple_whitespaces(self,text):
        """ Remove whitespaces from text """
        return re.sub(r'\s+', ' ', text, flags=re.I)

   
    def remove_trailing_whitespaces(self,text):
        """ Remove whitespaces from beggining and ending text """
        return text.strip()
     
    def lemmatize(self, text):
        """ Lemmatize words in text """          
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()
        text = stemmer.stem(text)
        return text

    def remove_stopwords(self, text):
        """ Remove stopwords from text 
        
        :param text: text
        :return: filtered stopwords from text
        """
        tokens = tweet_tokenizer.tokenize(text)
        filtered_tokens = [token for token in tokens if token.lower() not in self.stopwords]
        text = " ".join(filtered_tokens)
        return text
    
    def fix_slang(self, text, slang):
        """ Fix slang words from text 
        
        :param text: text
        :return: filtered slang from text
        """
        tokens = tweet_tokenizer.tokenize(text)

        filtered_tokens = []
        gotcha = 0

        for token in tokens:
            for index, row in slang.iterrows():
                if token == row["slang"]:
                    filtered_tokens.append(row["formal"])
                    gotcha = 1
        
            if gotcha == 0:
                filtered_tokens.append(token)
            else:
                gotcha = 0
            
        text = " ".join(filtered_tokens)

        return text
       
    def clean_text(self, text, slang):
        """ Generate a clean textual content"""
        text = self.remove_url(text)
        text = self.lowercase(text)
        text = self.remove_newlines(text)
        text = self.remove_tabs(text)
        text = self.remove_digits(text)
        text = self.replace_special_chars(text)
        text = self.remove_multiple_whitespaces(text)
        text = self.remove_trailing_whitespaces(text)        
        text = self.fix_slang(text, slang)
        text = self.lemmatize(text)
        text = self.remove_stopwords(text)        
        return text
    
    def clean_text_ws(self, text, slang):
        """ Generate a clean textual content"""
        text = self.remove_url(text)
        text = self.lowercase(text)
        text = self.remove_newlines(text)
        text = self.remove_tabs(text)
        text = self.remove_digits(text)
        text = self.replace_special_chars(text)
        text = self.remove_multiple_whitespaces(text)
        text = self.remove_trailing_whitespaces(text)        
        text = self.fix_slang(text, slang)     
        text = self.lemmatize(text)
        return text
    
    def process_text(self, data, slang):
        result_sentence=[]
        result_sentence_ws=[]
        result_tokens=[] 
        for sentence in data:
            clean_sentence = self.clean_text(sentence, slang)
            result_sentence.append(clean_sentence)
        return  result_sentence

In [ ]:
processed_data = tweets
prep = Preprocessing()
data_clean = prep.process_text(processed_data["tweet"], slang)

processed_data['clean_sentences'] = data_clean

processed_data


In [ ]:
processed_data.to_csv('/Data/tweet_processed.csv',  index = False)